In [98]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam, RMSprop, Adadelta, Adagrad
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
import numpy as np

X_train = np.load("X_train_1375_0.npy")
Y_train = np.load("Y_train_1375_0.npy")

X_dev = np.load("X_dev_1375.npy")
Y_dev = np.load("Y_dev_1375.npy")

In [3]:

X_train.shape

(545, 5511, 101)

In [113]:
# GRADED FUNCTION: model

def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    ### START CODE HERE ###
    
    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=2, strides=1)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.5)(X)                                 # dropout (use 0.8)

        # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=12, strides=4)(X)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.5)(X)                                 # dropout (use 0.8)
    
    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.5)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.5)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.5)(X)                                  # dropout (use 0.8)
    
    # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)
    
    return model

In [114]:
Tx = 5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
model = model(input_shape = (Tx, n_freq))

In [115]:
model.summary()

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 5511, 101)         0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 5510, 196)         39788     
_________________________________________________________________
batch_normalization_118 (Bat (None, 5510, 196)         784       
_________________________________________________________________
activation_58 (Activation)   (None, 5510, 196)         0         
_________________________________________________________________
dropout_148 (Dropout)        (None, 5510, 196)         0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 1375, 196)         461188    
_________________________________________________________________
batch_normalization_119 (Bat (None, 1375, 196)         784

In [116]:
opt = RMSprop(lr=0.1, rho=0.9, epsilon=1e-07)
#opt = Adadelta(lr=0.01, rho=0.95, epsilon=1e-07)
#opt = Adagrad(0.01, 0.1,  1e-6)
#opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
model_total = model.fit(X_train, Y_train, batch_size = 10, epochs=30, validation_data=(X_dev, Y_dev))

Train on 545 samples, validate on 135 samples
Epoch 1/30
545/545 [==============================] - 917s 2s/step - loss: 0.3736 - accuracy: 0.9101 - val_loss: 0.3192 - val_accuracy: 0.9335
Epoch 2/30
180/545 [========>.....................] - ETA: 12:01 - loss: 0.3634 - accuracy: 0.9028

In [ ]:
model.save('ml2_tr.h5')

In [ ]:
X_dev.shape

In [ ]:
loss, acc = model.evaluate(X_dev, Y_dev)
print("Dev set accuracy = ", acc)

In [ ]:
def detect_triggerword(filename):
    plt.subplot(2, 1, 1)

    x = graph_spectrogram(filename)
    print(x)
    print(x.shape)
    # the spectogram outputs (freqs, Tx) and we want (Tx, freqs) to input into the model
    x  = x.swapaxes(0,1)
    print(x.shape)
    x = np.expand_dims(x, axis=0)
    print(x.shape)
    predictions = model.predict(x)
    print(predictions)
    print(Y_dev)
    plt.subplot(2, 1, 2)
    plt.plot(predictions[0,:,0])
    plt.ylabel('probability')
    plt.show()
    return predictions

In [ ]:
chime_file = "chime.wav"
def chime_on_activate(filename, predictions, threshold):
    audio_clip = AudioSegment.from_wav(filename)
    chime = AudioSegment.from_wav(chime_file)
    Ty = predictions.shape[1]
    # Step 1: Initialize the number of consecutive output steps to 0
    consecutive_timesteps = 0
    # Step 2: Loop over the output steps in the y
    for i in range(Ty):
        # Step 3: Increment consecutive output steps
        consecutive_timesteps += 1
        # Step 4: If prediction is higher than the threshold and more than 75 consecutive output steps have passed
        if predictions[0,i,0] > threshold and consecutive_timesteps > 75:
            # Step 5: Superpose audio and background using pydub
            audio_clip = audio_clip.overlay(chime, position = ((i / Ty) * audio_clip.duration_seconds)*1000)
            # Step 6: Reset consecutive output steps to 0
            consecutive_timesteps = 0
        
    audio_clip.export("chime_output.wav", format='wav')

In [ ]:
import numpy as np
from pydub import AudioSegment
import random
import sys
import io
import os
import glob
import IPython
from td_utils import *
%matplotlib inline

filename = "train_2.wav"
prediction = detect_triggerword(filename)
chime_on_activate(filename, prediction, 0.5)
IPython.display.Audio("./chime_output.wav")